## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

# Carregando os dados

In [50]:
%matplotlib inline
import numpy as np
from sklearn.preprocessing import LabelEncoder
import mne
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn import svm
from copy import deepcopy
from scipy.io import loadmat
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFECV

Carregando dados

In [51]:
data = loadmat(f"../../dataset/beta/S12.mat")['data'][0][0]
eeg_data = data[0]
eeg = eeg_data.reshape(eeg_data.shape[0], eeg_data.shape[1], eeg_data.shape[2] * eeg_data.shape[3])
labels = np.array(list(data[1]['freqs'][0][0].flatten()) * 4)
print(eeg.shape, labels.shape)

(64, 750, 160) (160,)


In [52]:
data_correct = eeg.swapaxes(0, 2)
data_correct = data_correct.swapaxes(1, 2)
print(data_correct.shape)

(160, 64, 750)


Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [53]:
dataset = deepcopy(data_correct)
print(dataset.shape)

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# Estimando o ruído de fundo
estimated_background_noise = []

for trial in dataset: #para cada sinal   
    noise_power = [] # armazena uma lista com as médias de potência para cada canal

    for electrode in trial:
        psd = np.abs(np.fft.fft(electrode)) ** 2

        # média da potência nos intervalos de tempo sem estímulo
        base_power = np.mean(psd[base_start:base_end])
        rest_power = np.mean(psd[rest_start:rest_end])
        mean_noise_power = (base_power + rest_power) / 2

        noise_power.append(mean_noise_power)

    #média das médias de potência de todos os canais para estimar o ruído de fundo
    estimated_background_noise.append(np.mean(noise_power))
# end


background_noise = np.array(estimated_background_noise)
print(background_noise.shape)
print(np.mean(background_noise))

(160, 64, 750)
(160,)
28376881.989714384


Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [54]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for x in range(dataset.shape[0]):
    samples = []
    
    for channel_data in dataset[x, :, :]:
        fft_result = np.fft.fft(channel_data)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
        # encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)
        samples.append(target_amplitudes_trial)
    target_amplitudes.append(samples)
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [55]:
def narrow_band_SNR(target_amplitudes, estimated_background_noise):
  target_amplitudes_adjusted = np.abs(target_amplitudes - estimated_background_noise)
  return 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)

def wide_band_SNR(target_amplitudes, estimated_background_noise):
  target_amplitudes_adjusted = np.abs(target_amplitudes - estimated_background_noise)
  total_power = np.sum(target_amplitudes_adjusted)
  return 10 * np.log10(target_amplitudes_adjusted / total_power)

# Remoção manual de caracteristicas

In [56]:
X = []
y = labels

for i, trial in enumerate(dataset):
  X.append([
    narrow_band_SNR(target_amplitudes[i], background_noise[i]),
    wide_band_SNR(target_amplitudes[i], background_noise[i])
  ])
  
X = np.array(X)
X = X.swapaxes(1, 2)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2]*X.shape[3])
print(X.shape)

(160, 64, 80)


In [57]:
# Remoção
# opções: ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2'] -> [47, 53, 54, 55, 56, 57, 60, 61, 62]
Xmanual = deepcopy(X)
Xmanual = Xmanual[:,[47, 53, 54, 55, 56, 57, 60, 61, 62],:]
Xmanual = Xmanual.reshape(Xmanual.shape[0], Xmanual.shape[1]*Xmanual.shape[2])

#preparing
Xmanual = StandardScaler().fit_transform(Xmanual)
ym = LabelEncoder().fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(Xmanual, ym, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

Acurácia: 0.025


c:\Users\gregu\Projetos\SSVEP\r_padrao\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


# Remoção automática de caracteristicas

In [58]:
X = []
y = labels

for i, trial in enumerate(dataset):
  X.append([
    narrow_band_SNR(target_amplitudes[i], background_noise[i]),
    wide_band_SNR(target_amplitudes[i], background_noise[i])
  ])
  
X = np.array(X)
X = X.swapaxes(1, 2)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2]*X.shape[3])
print(X.shape)

(160, 64, 80)


In [59]:
# preparing
Xauto = deepcopy(X)
Xauto = Xauto.reshape(Xauto.shape[0], Xauto.shape[1]*Xauto.shape[2])
Xauto = StandardScaler().fit_transform(Xauto)
ya = LabelEncoder().fit_transform(y)
print(Xauto.shape, ya.shape)

#rfe
rfe = RFECV(svm.SVC(kernel="linear"), step=0.0001, min_features_to_select=1, cv=3)
X_final = rfe.fit_transform(Xauto, ya)
print(X_final.shape)

#svm
x_train, x_test, y_train, y_test = train_test_split(X_final, ya, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print("accuracy", accuracy_score(y_test, y_pred))
print("f1_score", f1_score(y_test, y_pred, average="weighted"))

(160, 5120) (160,)
(160, 183)


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.